In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px
import requests

In [2]:
from datetime import datetime

In [ ]:
# Leiskite vartotojui nurodyti iki trijų metreologinių stočių, ir dvi datas : nuo, iki.
# Iš meto.lt per API išgaukite istorinius duomenis nurodytame laikotarpyje,
# ir atvaizduokite grafiškai - oro temperatūrą bei vėjo greitį.

# https://docs.streamlit.io/develop/api-reference/widgets/st.multiselect


In [ ]:
uzklausa_miestas = input("iveskite miestu pavadinimus (max 3):")
nuo = input('nuo kada (yyyy-mm-dd)?')
iki = input('nuo kada (yyyy-mm-dd)?')

miestai = uzklausa_miestas.split()

In [ ]:
url1 = 'https://api.meteo.lt/v1/stations'

page = requests.get(url1)
r = page.json()
df = pd.DataFrame(data=r)
codes = df.loc[:,'code'].values

print(codes)

In [ ]:
url = 'https://api.meteo.lt/v1/stations'
pg = requests.get(url)
r = pg.json()
stotys = {}
for i in r:
    stotys[i['name']] = i['code']
    # codes[i['code']] = i['name']
# print(stotys)
n = list(stotys.keys())
kodai = list(stotys.values())

# print(n)
# print(kodai)
for i, k in enumerate(n):
    print(f'{i}: {k}')


In [ ]:
stations = input('Įveskite stoties numerį(-ius) (iki trijų numerių, atskirtų kableliais)')
st_idx = list(map(int, stations.split(',')))
print('Jūs pasirinkote šias stotis:')
sel_sts = [n[i] for i in st_idx]
sel_kodai = [kodai[i] for i in st_idx]
print(sel_sts)
print(sel_kodai)

In [ ]:
print("pasirinkite laikotarpi (max menuo)")

In [10]:
nuo = input('nuo kada (yyyy-mm-dd)?')
iki = input('nuo kada (yyyy-mm-dd)?')


In [ ]:
# start date
start_date = datetime.strptime(nuo, "%Y-%m-%d")
end_date = datetime.strptime(iki, "%Y-%m-%d")

# difference between each date. D means one day
D = 'D'

date_list = pd.date_range(start_date, end_date, freq=D)
print(f"Creating list of dates starting from {start_date} to {end_date}")
# print(date_list)

# if you want dates in string format then convert it into string
print(date_list.strftime("%Y-%m-%d"))



In [ ]:
from datetime import date, timedelta
from dateutil import parser
# nuo = input('Įveskite datą YYYY-MM-DD formatu')
# iki = input('Įveskite datą YYYY-MM-DD formatu')

datos = []
laikai = []

start_date = parser.parse(nuo)
end_date = parser.parse(iki)
delta = timedelta(days=1)
while start_date <= end_date:
    print(start_date.strftime("%Y-%m-%d"))
    datos.append(str(start_date))
    start_date = start_date + delta
    
    
for i in datos:
    laikai.append(i[:10])
    
print(laikai)

In [ ]:
stotys = []
data = []
temperatura = []
vejas = []


for d in laikai:
    for s in sel_kodai:
        url2 = f'https://api.meteo.lt/v1/stations/{s}/observations/{d}'
        observations = requests.get(url2).json()['observations']
        for o in observations:
            stotys.append(s)
            data.append(o['observationTimeUtc'])
            temperatura.append(o['airTemperature'])
            vejas.append(o['windSpeed'])

print(temperatura)
print(vejas)

ndf = pd.DataFrame(data)
ndf['Kodas'] = pd.DataFrame(data=stotys)
ndf['Temperatūros'] = pd.DataFrame(data=temperatura)
ndf['Data'] = pd.DataFrame(data=data)
ndf['vejasSpeed'] = pd.DataFrame(data=vejas)

ndf['Datos'] = pd.to_datetime(ndf['Data'])
ndf['Laikai'] = pd.to_datetime(ndf['Data']).dt.time

fig, axes = plt.subplots(figsize=(8, 4.5))

axes.set_title('Temperatūros')
ax = sns.lineplot(data=ndf,x = 'Datos', y='Temperatūros', ax=axes, hue='Kodas', palette='Set2', style='Kodas')
axes.set(xlabel='Laikas',ylabel='Temperatura')
# axes.set_xticks(data)
axes.tick_params(axis='x', rotation=90, labelsize=6)
for container in ax.containers:
    ax.bar_label(container)
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(8, 4.5))

axes.set_title('Vėjas')
ax = sns.lineplot(data=ndf,x = 'Datos', y='vejasSpeed', ax=axes, hue='Kodas', palette='Set2', style='Kodas')
axes.set(xlabel='Laikas',ylabel='Vėjo greitis m/s')
# axes.set_xticks(data)
axes.tick_params(axis='x', rotation=90, labelsize=6)
for container in ax.containers:
    ax.bar_label(container)
plt.show()

In [ ]:
# https://data.gov.lt/datasets/2638/models/VietovesTemperaturaDregme/
# nuskaitykite šiuos duomenis (atsisiuntę kaip CSV ar kitu jums priimtinu būdu)
# atvaizduokite su scatter_mapbox, kuriose Lietuvos vietose yra įrengti registratoriai.
# atvaizduokite vieno bet kurio vietos registratoriaus užfiksuotas temperatūras (istorinius duomenis)
# 8 cm gylyje, paviršiuje, 15 cm aukštyje.

# zorder
# .plot(x, y, zorder=5)

In [ ]:
vdtd = pd.read_csv('../../../eismas/VietovesTemperaturaDregme.csv')[[
    'atvejo_id',
    'vietoves_registratoriaus_id',
    'laikas',
    'temperatura_8cm_gylyje_dirvozemyje',
    'temperatura_dirvozemio_pavirsiuje',
    'temperatura_15cm_virs_dirvozemio',
    'platuma',
    'ilguma'
]]
vdtd.head()

In [ ]:
vdtd.info()

In [ ]:
vdtd_gr = vdtd.groupby(['ilguma', 'platuma']).size().rename('count').reset_index()
vdtd_gr.head()
print(len(vdtd_gr))

In [ ]:
fig = px.scatter_mapbox(data_frame=vdtd, lon='ilguma', lat='platuma', zoom=5, title='registratoriai')
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(width=800, height=600)
fig.show()

In [17]:
vdtd['laikas'] = pd.to_datetime(vdtd['laikas'])

In [ ]:
fig = px.scatter_mapbox(data_frame=vdtd, lon='ilguma', lat='platuma', zoom=5, title='registratoriai')
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
regs_id = list(set(list(vdtd['vietoves_registratoriaus_id'])))
print(regs_id)

In [ ]:
reg1 = regs_id[0]
reg1df = vdtd[vdtd['vietoves_registratoriaus_id']==reg1]
print(reg1)

In [9]:
reg7 = regs_id[6]
reg7df = vdtd[vdtd['vietoves_registratoriaus_id']==reg7]

In [ ]:
fig, axes = plt.subplots(figsize=(8,4.5))

axes.set_title(f'Temperatūros registratoriaus {reg1}')
sns.scatterplot(data=reg1df,x = 'laikas', y='temperatura_8cm_gylyje_dirvozemyje', ax=axes,label='-8cm', zorder=5)
sns.scatterplot(data=reg1df,x = 'laikas', y='temperatura_15cm_virs_dirvozemio', ax=axes,label='15cm')
sns.scatterplot(data=reg1df,x = 'laikas', y='temperatura_dirvozemio_pavirsiuje', ax=axes,label='0cm')
axes.set(xlabel='Data ir laikas',ylabel='Temperatūra')
axes.legend()
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(8,4.5))

axes.set_title(f'Temperatūros registratoriaus {reg7}')
sns.lineplot(data=reg7df,x = 'laikas', y='temperatura_8cm_gylyje_dirvozemyje', ax=axes,label='-8cm', zorder=5)
sns.lineplot(data=reg7df,x = 'laikas', y='temperatura_15cm_virs_dirvozemio', ax=axes,label='15cm')
sns.lineplot(data=reg7df,x = 'laikas', y='temperatura_dirvozemio_pavirsiuje', ax=axes,label='0cm')
axes.set(xlabel='Data ir laikas',ylabel='Temperatūra')
axes.legend()
plt.show()